In [1]:
import pandas as pd
import requests
import os

In [4]:
file_csv = '~/code/harlqeuinht/which_horse/raw_data/combined_flat2_csv.csv'

In [42]:
df = pd.read_csv(file_csv)

/tmp/ipykernel_1478/2113571882.py:1: DtypeWarning: Columns (22,53,62,71,80,89,98,107,116) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_csv)


In [125]:
columns_to_drop = df.filter(regex='[678]').columns
columns_to_drop = columns_to_drop.drop('bet365_odds')
df = df.drop(columns=(columns_to_drop))

Create a DataFrame which stores the meeting location name and long & lat. Note this has been done on the sample dataset, not the full dataset

### Locations API ###

In [192]:
location_names = sorted(df['meeting_name'].unique())
#location_names = [location for location in location_names if location.lower() != 'newmarket (july)']
locations_df = pd.DataFrame(location_names, columns=['meeting_name'])
locations_df['location_names_cleaned'] = locations_df['meeting_name'].replace({'BANGOR-ON-DEE':'BANGOR', 'NEWMARKET (JULY)':'NEWMARKET', ' ':'_'})
locations_df['location_names_cleaned'] = locations_df['location_names_cleaned'].str.replace(' ', '_')

Temporarily reduce the dataframe to 5 rows

In [193]:
locations_df

,meeting_name,location_names_cleaned
0,AINTREE,AINTREE
1,ASCOT,ASCOT
2,AYR,AYR
3,BANGOR-ON-DEE,BANGOR
4,BATH,BATH
5,BEVERLEY,BEVERLEY
6,BRIGHTON,BRIGHTON
7,CARLISLE,CARLISLE
8,CATTERICK,CATTERICK
9,CHELMSFORD,CHELMSFORD


Create API endpoint URLs

In [198]:
api_urls = []
all_results = []
for index, location in enumerate(locations_df['location_names_cleaned']):
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
    api_key = os.environ['KEY']
    params = f'address={location}+racecourse&components=country:GB&key={api_key}'
    endpoint = f'{base_url}{params}'
    results = requests.get(endpoint).json()
    results_dict = {'location':location,
                    'co_ords':{'lat':results['results'][0]['geometry']['location']['lat'],
                               'lng':results['results'][0]['geometry']['location']['lng']
                              }
                   }
    all_results.append(results_dict)
    locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
    locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']


In [199]:
locations_df

,meeting_name,location_names_cleaned,lat,lng
0,AINTREE,AINTREE,53.473476,-2.954128
1,ASCOT,ASCOT,51.412167,-0.679485
2,AYR,AYR,55.467721,-4.612545
3,BANGOR-ON-DEE,BANGOR,53.004035,-2.909421
4,BATH,BATH,51.418670,-2.407209
5,BEVERLEY,BEVERLEY,53.843136,-0.458438
6,BRIGHTON,BRIGHTON,50.829821,-0.112602
7,CARLISLE,CARLISLE,54.858994,-2.928866
8,CATTERICK,CATTERICK,54.388054,-1.649181
9,CHELMSFORD,CHELMSFORD,51.842047,0.512814


### Weather API ###

In [200]:
sample_df = df.head(5)
sample_df

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_4_dsr,run_5_raw_post_race_rating_int,run_5_raw_post_race_rating_symbol,run_5_final_rating_int,run_5_race_type,run_5_race_class,run_5_race_class_normalised,run_5_track_type,run_5_win_lose,run_5_dsr
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,305.0,59.0,NaN,66.0,FLAT,5,Handicap,TURF,0.0,342.0
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,226.0,60.0,NaN,71.0,FLAT,6,Handicap,TURF,0.0,279.0
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,371.0,56.0,NaN,68.0,FLAT,3,Handicap,TURF,0.0,383.0
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,47.0,59.0,NaN,69.0,FLAT,6,Handicap,TURF,0.0,272.0
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,417.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [202]:
sample_df = df.head(5)
temp = pd.merge(df,locations_df, on = 'meeting_name')
temp

,meeting_id,meeting_name,date,country_code,event_number,distance,distance_unit,distance_furlongs,distance_raw_furlongs,race_class,...,run_5_final_rating_int,run_5_race_type,run_5_race_class,run_5_race_class_normalised,run_5_track_type,run_5_win_lose,run_5_dsr,location_names_cleaned,lat,lng
0,3668,AYR,2020-07-20,GBR,1,1100,yards,5f,5.0,6,...,66.0,FLAT,5,Handicap,TURF,0.0,342.0,AYR,55.467721,-4.612545
1,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,71.0,FLAT,6,Handicap,TURF,0.0,279.0,AYR,55.467721,-4.612545
2,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,68.0,FLAT,3,Handicap,TURF,0.0,383.0,AYR,55.467721,-4.612545
3,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,69.0,FLAT,6,Handicap,TURF,0.0,272.0,AYR,55.467721,-4.612545
4,3668,AYR,2020-07-20,GBR,7,1760,yards,8f,8.0,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AYR,55.467721,-4.612545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282001,37996,NEWMARKET,2023-09-28,GBR,3,1320,yards,6f,6.0,2,...,81.0,FLAT,1.0,Listed,TURF,0.0,145.0,NEWMARKET,52.238014,0.374746
282002,37996,NEWMARKET,2023-09-28,GBR,2,1760,yards,8f,8.0,4,...,70.0,FLAT,5.0,Novice,TURF,0.0,105.0,NEWMARKET,52.238014,0.374746
282003,37996,NEWMARKET,2023-09-28,GBR,2,1760,yards,8f,8.0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEWMARKET,52.238014,0.374746
282004,37996,NEWMARKET,2023-09-28,GBR,3,1320,yards,6f,6.0,2,...,105.0,FLAT,1.0,Group 3,TURF,0.0,392.0,NEWMARKET,52.238014,0.374746


In [ ]:
for index, row in sample_df.iterrows():
    lat = sample_df['lat']
    lng = sample_df['lng']
    date = sample_df['date']

In [ ]:
weather_temp = []
weather_rain = []
weather_wind = []

all_results = []
for index, location in enumerate(locations_df['location_names_cleaned']):
    base_url = 'https://archive-api.open-meteo.com/v1/archive?'
    params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
    endpoint = f'{base_url}{params}'
    results = requests.get(endpoint).json()
    results_dict = {'location':location,
                    'co_ords':{'lat':results['results'][0]['geometry']['location']['lat'],
                               'lng':results['results'][0]['geometry']['location']['lng']
                              }
                   }
    all_results.append(results_dict)
    locations_df.loc[index, 'lat'] = results['results'][0]['geometry']['location']['lat']
    locations_df.loc[index, 'lng'] = results['results'][0]['geometry']['location']['lng']


In [ ]:
base_url = 'https://archive-api.open-meteo.com/v1/archive?'
longitude = '-1.597'
latitude = '52.279'
start_date = '2020-01-01'
end_date = '2020-01-07'
params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
endpoint = f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&{params}'
endpoint

print(requests.get(endpoint).json())

In [179]:
weather_result = requests.get(endpoint).json()
precipitation = weather_result['daily']['precipitation_sum']
wind = weather_result['daily']['wind_speed_10m_max']
temp = weather_result['daily']['temperature_2m_mean']

In [160]:
base_url = 'https://archive-api.open-meteo.com/v1/archive?'
longitude = '-1.597'
latitude = '52.279'
start_date = '2020-01-01'
end_date = '2020-01-07'
params = 'daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'
endpoint = f'{base_url}&latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&{params}'
endpoint

print(requests.get(endpoint).json())


{'latitude': 52.26713, 'longitude': -1.6636047, 'generationtime_ms': 0.33104419708251953, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 51.0, 'daily_units': {'time': 'iso8601', 'temperature_2m_mean': '°C', 'precipitation_sum': 'mm', 'rain_sum': 'mm', 'wind_speed_10m_max': 'km/h', 'wind_direction_10m_dominant': '°'}, 'daily': {'time': ['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07'], 'temperature_2m_mean': [5.9, 8.3, 7.0, 5.8, 6.1, 6.9, 9.3], 'precipitation_sum': [0.0, 0.0, 4.8, 0.0, 0.0, 1.4, 0.1], 'rain_sum': [0.0, 0.0, 4.8, 0.0, 0.0, 1.4, 0.1], 'wind_speed_10m_max': [15.3, 27.1, 23.0, 27.3, 22.7, 28.3, 31.3], 'wind_direction_10m_dominant': [171, 204, 280, 267, 222, 211, 220]}}


In [161]:
endpoint

'https://archive-api.open-meteo.com/v1/archive?&latitude=52.279&longitude=-1.597&start_date=2020-01-01&end_date=2020-01-07&daily=temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,wind_direction_10m_dominant'

In [58]:
latitude = locations_df['latitude']
longitude = locations_df['longitude']

KeyError: 'latitude'

In [ ]:
# This code works and returns a 200 code for current weather data
response = requests.get('http://api.weatherapi.com/v1/current.json?key=581fec608fba4a5699790722240703&dt=2020-01-01&q=London')
response

In [ ]:
print(response.json())

In [3]:
os.environ['KEY']

'yscbfocbiybdiuhabpiusxhb'

In [1]:
print("hey")

hey
